In [ ]:
import base64
import requests

# Your Client ID and Client Secret
client_id = 'c6a34d0e2f2940d099b19deb4841bc1b'
client_secret = ''

# Encode as Base64
credentials = f"{client_id}:{client_secret}"
encoded_credentials = base64.b64encode(credentials.encode()).decode()

# Request access token
token_url = 'https://accounts.spotify.com/api/token'
headers = {
    'Authorization': f'Basic {encoded_credentials}',
    'Content-Type': 'application/x-www-form-urlencoded'
}
data = {'grant_type': 'client_credentials'}

response = requests.post(token_url, headers=headers, data=data)
access_token = response.json().get('access_token')

In [3]:
import pandas as pd

def get_artists_from_albums(access_token, popularity_threshold=30, limit=50):
    headers = {'Authorization': f'Bearer {access_token}'}
    search_url = 'https://api.spotify.com/v1/search'
    params = {
        'q': 'tag:hipster',
        'type': 'album',
        'limit': limit
    }

    response = requests.get(search_url, headers=headers, params=params)
    albums = response.json().get('albums', {}).get('items', [])
    unique_artists = {}
    for album in albums:
        for artist in album['artists']:
            artist_id = artist['id']
            artist_name = artist['name']
            artist_url = f'https://api.spotify.com/v1/artists/{artist_id}'
            artist_response = requests.get(artist_url, headers=headers)
            if artist_response.status_code == 200:
                artist_data = artist_response.json()
                if artist_data['popularity'] < popularity_threshold:
                    unique_artists[artist_id] = {
                                'name': artist_name,
                                'id': artist_id,
                                'popularity': artist_data['popularity'],
                                'followers': artist_data['followers']['total']
                            }

    return pd.DataFrame(unique_artists.values())

num_searches = 1
df_artists = pd.DataFrame()
for _ in range(num_searches):
    df_artists = pd.concat([df_artists, get_artists_from_albums(access_token=access_token)])
print(df_artists.head(20))

                name                      id  popularity  followers
0       One Eyed Men  329EuuPhDjYT3tgnChtpWI          10         42
1    Dj Krankenwagen  5lugcmzGeaXMBdrizEJK4S           9        451
2   Cheb Aziz Staifi  2JLRf4pU6gvzCbicbQSock          11        547
3          Aesthetix  7CfUfh5xgpw8Pj4gciQBls           4         33
4               +TEK  0tdghWSPQoEByqbgBlAwyJ          17        576
5              Chopo  6aGPZygnKSLg5UCBQD0eJn          11        552
6           Number Ö  4EyVUboKgjF1ePnTWLOhaP           4        150
7               Nani  7rlKkIi4IB5tLACIABV3iX          25       4905
8         Nd Githuka  7fqwiukJ0n2HU6e0muFwby          20       6120
9            Mad Fox  4H7IHUXblLFYpGuL0xOqSJ           1       1622
10        Yakuza Mvp  20S0ZdD0VKEOqhkzCPLiKe           7         37
11  Batista Cadillac  5w2K4e2FXj7bbPao5T0z0R          23       3280
12           Jamirko  6S5Op08abwjGf31EvVWomN           1          3
13              H'Mx  4pJhHCpPOUARcOT8tnIdsj    

In [4]:
redirect_uri = "http://localhost:8888/callback"
auth_url = (
    f"https://accounts.spotify.com/authorize?client_id={client_id}"
    f"&response_type=code"
    f"&redirect_uri={redirect_uri}"
    f"&scope=user-top-read"
)
print("Visit this URL to authorize:", auth_url)


Visit this URL to authorize: https://accounts.spotify.com/authorize?client_id=c6a34d0e2f2940d099b19deb4841bc1b&response_type=code&redirect_uri=http://localhost:8888/callback&scope=user-top-read


In [5]:
auth_header = base64.b64encode(f"{client_id}:{client_secret}".encode()).decode()

token_url = "https://accounts.spotify.com/api/token"
token_data = {
    "grant_type": "authorization_code",
    "code": "AQDsd5bWWLXGh2h6BC5PXoFPO1_IO9w-YVEzfw9sakKe9P-rj4zVfmHPJ7SnCLceyq5yuQxJRQ6yFOkU85JiemQuViDGnkd6doA5mtnioif5OuHVgb_wwm3KXCWnr0VSTY6UAn3LS9lCC8eqV3V5n2l5-W_AF6uLLy5Nwe9xujJ5-Xp_N7Ic14zAvuP3ZRQh2w",
    "redirect_uri": redirect_uri,
}
token_headers = {
    "Authorization": f"Basic {auth_header}",
    "Content-Type": "application/x-www-form-urlencoded",
}

response = requests.post(token_url, data=token_data, headers=token_headers)
token_info = response.json()

access_token = token_info.get("access_token")

In [6]:
url = "https://api.spotify.com/v1/me/top/artists"
headers = {
    "Authorization": f"Bearer {access_token}"
}
params = {
    "time_range": "medium_term",
    "limit": 10
}

response = requests.get(url, headers=headers, params=params)
top_artists = response.json()

# print(top_artists)
for artist in top_artists['items']:
    print(f"Name: {artist['name']}, Spotify ID: {artist['id']}")

KeyError: 'items'

In [ ]:
import json

recommended_artists = {}

for artist in top_artists['items']:
    url = f"https://api.spotify.com/v1/artists/{artist['id']}/related-artists"
    headers = {
        "Authorization": f"Bearer {access_token}"
    }

    response = requests.get(url, headers=headers)
    # print(json.dumps(response.json(), indent=4))

    if response.status_code == 200:
        related_artists = response.json().get('artists', [])
        
        popularity_threshold = 30

        filtered_artists = [
            artist for artist in related_artists if (artist['popularity'] < popularity_threshold) & (artist not in top_artists['items'])
        ]

        for artist in filtered_artists:
            print(artist['name'])
    else:
        print(f"Error: Status code 200.")






Mark Hollis
Holger Czukay
Matching Mole
Annette Peacock
White Noise
Pearls Before Swine
Slapp Happy
Harmonia
The Lounge Lizards
Trees
helen island
Jonnine
Lolina
great area
Martha Skye Murphy
Spivak
pigbaby
Martyna Basta
Bianca Scout
Vanessa Bedoret
great area
Gyeongsu
Jonnine
Au Pairs
The Monochrome Set
Pere Ubu
The Soft Boys
Josef K
Swell Maps
The Boys Next Door
Buffalo Daughter
Sketch Show
Polaris
Takako Minekawa
mmm
